In [11]:
# testの空間への埋め込み

In [12]:
import sys
import os
import torch
import pandas as pd
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc
import json
from torchvision import transforms
import matplotlib.pyplot as plt
import glob
from PIL import Image


In [13]:
import importlib
import utils
importlib.reload(utils.util)

<module 'utils.util' from 'C:\\Users/yuuta/Documents/fashion\\utils\\util.py'>

In [14]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [15]:
positive_path = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [16]:
targets = []
p_cnt = 0
for fp in positive_path:
    json_dict = open_json(fp)
    items = filter_basic_items(json_dict["items"])
    attributes = []
    vectors = []
    for item in items:
        try:
            itemId = str(item["itemId"])
        except Exception as e:
            print(fp, " : ", e)
            continue

        targets.append(itemId)

In [17]:
from model_learning.model_structure.image_encoder import ImageEncoderV2

model = ImageEncoderV2().to(device)

model.load_state_dict(
    torch.load(
        "C:/Users/yuuta/Documents/fashion/model_learning/compatibility/models/triplet-image-2024-01-09.pth"
    )
)
def infer(input_img):
    with torch.no_grad():
        pred = model(input_img)
    return pred

In [18]:
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)


def to_image(itemId):
    image_path = f"C:/Users/yuuta/Documents/fashion/data/images/{itemId}.jpg"
    image = Image.open(image_path)
    if image.mode != "RGB":
        image = image.convert("L")
        image = Image.merge("RGB", [image] * 3)

    image = transform(image)
    return image


ids = []
# captions = []
images = []
vectors = {}
BATCH_SIZE = 32
cnt = 0

In [19]:
# vectors = []
for i in range(0, len(targets), BATCH_SIZE):
    if i % 100 == 0:
        print(f"\r{i * 100 / len(targets)} %", end='')
    s = i
    e = min(i + BATCH_SIZE, len(targets))

    images = torch.stack(list(map(to_image, targets[s:e]))).to(torch.device("cuda"))
    pred = infer(images)
    pred_ids = targets[s:e]
    for id, vector in zip(targets[s:e], pred):
        vectors[id] = vector.to("cpu").tolist()


99.83673957825816 %%

In [20]:
with open(f"C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json", "w") as f:
    json.dump(vectors, f)